In [1]:
using PlotlyJS
using XLSX
include("../model/utils.jl")
include("../model/unit_commitment.jl")
include("../model/economic_dispatch.jl")
include("./plotting.jl")
include("./processing.jl")
# XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))

WebIO._IJuliaInit()

calculate_reserve_gcd_KPI (generic function with 1 method)

In [2]:
folder_path = joinpath("..","output", "solutions_v2.4")
solution_folders = ["n_7", "n_228", "n_259", "n_289"]
# solution_folders = ["n_6", "n_7", "n_9", "n_10"]
# solution_folders = ["n_228"]

keys = [:demand, :generation, :storage, :reserve, :energy_reserve, :scalar]
s_uc = [parquet_to_solution("s_uc", joinpath(folder_path, s)) for s in solution_folders]
s_ed = [parquet_to_solution("s_ed", joinpath(folder_path, s)) for s in solution_folders]
s_uc = NamedTuple(k => vcat([s[k] for s in s_uc if haskey(s, k)]...) for k in keys)
s_ed = NamedTuple(k => vcat([s[k] for s in s_ed if haskey(s, k)]...) for k in keys)
;

reading...
../output/solutions_v2.4/n_7/s_uc_demand.parquet
../output/solutions_v2.4/n_7/s_uc_generation.parquet
../output/solutions_v2.4/n_7/s_uc_storage.parquet
../output/solutions_v2.4/n_7/s_uc_reserve.parquet
../output/solutions_v2.4/n_7/s_uc_scalar.parquet
...done
reading...
../output/solutions_v2.4/n_228/s_uc_demand.parquet
../output/solutions_v2.4/n_228/s_uc_generation.parquet
../output/solutions_v2.4/n_228/s_uc_storage.parquet
../output/solutions_v2.4/n_228/s_uc_reserve.parquet
../output/solutions_v2.4/n_228/s_uc_scalar.parquet
...done
reading...
../output/solutions_v2.4/n_259/s_uc_demand.parquet
../output/solutions_v2.4/n_259/s_uc_generation.parquet
../output/solutions_v2.4/n_259/s_uc_storage.parquet
../output/solutions_v2.4/n_259/s_uc_reserve.parquet
../output/solutions_v2.4/n_259/s_uc_scalar.parquet
...done
reading...
../output/solutions_v2.4/n_289/s_uc_demand.parquet
../output/solutions_v2.4/n_289/s_uc_generation.parquet
../output/solutions_v2.4/n_289/s_uc_storage.parquet
.

In [3]:
gcdi_KPI = calculate_adecuacy_gcdi_KPI(s_ed, s_uc)
gcd_KPI = calculate_adecuacy_gcd_KPI(gcdi_KPI)
KPI_reserve = calculate_reserve_KPI(s_ed, s_uc)
gcdi_KPI_reserve = calculate_reserve_gcdi_KPI(KPI_reserve)
gcd_KPI_reserve = calculate_reserve_gcd_KPI(gcdi_KPI_reserve)
;

In [4]:
# thres =.001 # 1 Watt
# f_LOL(x,y) = 
#     (LLD_h=count(x.>thres),
#     # LOLP=count(x.>thres)/length(y)*100,
#     ENS_MWh = sum(x),
#     # LOL_percentage = sum(x)/(sum(y) + sum(x))*100,
#     demand_MWh = (sum(y) + sum(x)),
#     )
# f_CUR(x,y) =     
#     (CURD_h=count(x.>thres),
#     # CURP=count(x.>thres)/length(y)*100,
#     CUR_MWh = sum(x),
#     # CUR_percentage = sum(x)/(sum(y) + sum(x))*100,
#     RES_production_MWh = (sum(y) + sum(x)),
#     )

In [5]:
# group_by = [:configuration, :day]
# group_by_big = [:configuration, :day, :iteration]

# filter = in(["onshore_wind_turbine","small_hydroelectric","solar_photovoltaic", "net_generation"]).(s_ed.generation.resource)

# gdf_LOL = combine(groupby(s_ed.demand, group_by_big), [:LOL_MW, :demand_MW] => ((x,y)->f_LOL(x,y)) => AsTable)
# gdf_CUR = combine(groupby(s_ed.generation[filter,:], group_by_big), [:curtailment_MW, :production_MW] =>((x,y) -> f_CUR(x,y))=> AsTable)
# g_KPI = outerjoin(gdf_LOL, gdf_CUR, on = group_by_big)
# # if write
# #     XLSX.writetable(joinpath(folder_path,"LOL_CUR.xlsx"), "LOL_CUR" => change_type(LOL_CUR, Symbol, string))
# #     XLSX.writetable(joinpath(folder_path,"gdf_LOL_CUR.xlsx"), "gdf_LOL_CUR" =>  change_type(gdf_LOL_CUR, Symbol, string))
# # end
# ;

In [6]:
# # folowing leftjoin will repeat values right values for "iteration"
# s_ed_scalar = s_ed.scalar[:, Not(:termination_status)]
# leftjoin!(
#     s_ed_scalar, 
#     rename(s_uc.scalar[:, Not(:termination_status)], :objective_value =>:objective_value_uc),
#     on = setdiff(propertynames(s_ed_scalar), [:objective_value, :iteration]))
    
# # relative difference with respect to s_uc
# s_ed_scalar.Δobjective_value = s_ed_scalar.objective_value .- s_ed_scalar.objective_value_uc
# s_ed_scalar.Δobjective_value_relative = (s_ed_scalar.objective_value .- s_ed_scalar.objective_value_uc)./s_ed_scalar.objective_value_uc

# # relative difference with respect to ref_configuration = :base_ramp_storage_envelopes_up_0_dn_0
# ref_configuration = :base_ramp_storage_envelopes_up_0_dn_0
# ref_configuration = s_ed.scalar[s_ed.scalar.configuration .== ref_configuration, [:objective_value, :iteration, :day]]

# leftjoin!(s_ed_scalar, 
#     rename(ref_configuration, :objective_value => :objective_value_ref_conf), 
#     on = [:day, :iteration])

# s_ed_scalar.Δobjective_value_ref_conf = s_ed_scalar.objective_value .- s_ed_scalar.objective_value_ref_conf
# s_ed_scalar.Δobjective_value_relative_ref_conf = (s_ed_scalar.objective_value .- s_ed_scalar.objective_value_ref_conf)./s_ed_scalar.objective_value_ref_conf

# leftjoin!(g_KPI, s_ed_scalar, on = group_by_big)
# ;
# # # s_ed_scalar = leftjoin(left, right, on = setdiff(propertynames(left), [:objective_value, :iteration]))

In [7]:
# LOL = combine(groupby(gdf_LOL, group_by), [:LLD_h, :ENS_MWh] => ((x,y)->(LOLE = mean(x), EENS = mean(y))) => AsTable)
# CUR = combine(groupby(gdf_CUR, group_by), [:CURD_h, :CUR_MWh] => ((x,y)->(CURE = mean(x), ECUR = mean(y))) => AsTable)

# KPI = outerjoin(
#     LOL, 
#     CUR,
#     combine(groupby(g_KPI, group_by), [:objective_value, :Δobjective_value_relative_ref_conf] .=> mean .=> [:EOV, :EΔOV]),
#     on=[:configuration, :day])

# transform!(KPI, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(KPI, :mu)

# transform!(g_KPI, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(g_KPI, :mu)
# ;

In [8]:
# KPI_reserve = leftjoin(
#     s_ed.demand[!,union(group_by_big,[:hour, :demand_MW, :LOL_MW])], 
#     rename(s_uc.demand[!, union(group_by,[:hour, :demand_MW])], :demand_MW => :demand_uc_MW),
#     on = union(group_by,[:hour]))

# required = (KPI_reserve.demand_MW .+ KPI_reserve.LOL_MW.- KPI_reserve.demand_uc_MW)
# KPI_reserve.required_r_up_MW = required.*(required.>=0)
# KPI_reserve.required_r_dn_MW = - required.*(required.<0)

# KPI_reserve = outerjoin(
#     KPI_reserve,
#     combine(groupby(s_ed.generation, union(group_by_big, [:hour])), :production_MW => sum => :production_MW),
#     combine(groupby(s_ed.storage, union(group_by_big, [:hour])), [:charge_MW, :discharge_MW] .=> sum .=> [:charge_MW, :discharge_MW]),
#     on = union(group_by,[:iteration, :hour]))

# delivered = KPI_reserve.production_MW .+ KPI_reserve.discharge_MW .- KPI_reserve.charge_MW .- KPI_reserve.demand_uc_MW
# KPI_reserve.delivered_r_up_MW = delivered.*(delivered.>=0)
# KPI_reserve.delivered_r_dn_MW = -(delivered .+ KPI_reserve.LOL_MW).*(delivered.<0).*(required.<0)

# KPI_reserve.delivered_r_up_ratio  = KPI_reserve.delivered_r_up_MW./KPI_reserve.required_r_up_MW
# KPI_reserve.delivered_r_dn_ratio  = KPI_reserve.delivered_r_dn_MW./KPI_reserve.required_r_dn_MW
# ;

In [9]:
# gcdi_KPI_reserve  = combine(groupby(KPI_reserve, group_by_big), [:required_r_up_MW, :required_r_dn_MW, :delivered_r_up_MW, :delivered_r_dn_MW] .=> sum .=> [:required_r_up_MWh, :required_r_dn_MWh, :delivered_r_up_MWh, :delivered_r_dn_MWh])
# gcdi_KPI_reserve.delivered_r_up_ratio  = gcdi_KPI_reserve.delivered_r_up_MWh ./gcdi_KPI_reserve.required_r_up_MWh
# gcdi_KPI_reserve.delivered_r_dn_ratio  = gcdi_KPI_reserve.delivered_r_dn_MWh ./gcdi_KPI_reserve.required_r_dn_MWh
# # replace.(gcdi_KPI_reserve, Inf=>0)
# # replace!(gcdi_KPI_reserve.delivered_r_up_ratio, Inf => 1)
# # replace!(gcdi_KPI_reserve.delivered_r_dn_ratio, Inf => 1)

# gcd_KPI_reserve  = combine(groupby(gcdi_KPI_reserve, group_by), 
#     [:required_r_up_MWh, :required_r_dn_MWh, :delivered_r_up_MWh, :delivered_r_dn_MWh] .=> mean .=> [:E_required_r_up_MWh, :E_required_r_dn_MWh, :E_delivered_r_up_MWh, :E_deliverered_dn_MWh])
# # replace.(gcd_KPI_reserve, Inf=>0)
# gcd_KPI_reserve.delivered_r_up_ratio  = gcd_KPI_reserve.E_delivered_r_up_MWh ./gcd_KPI_reserve.E_required_r_up_MWh
# gcd_KPI_reserve.delivered_r_dn_ratio  = gcd_KPI_reserve.E_deliverered_dn_MWh ./gcd_KPI_reserve.E_required_r_dn_MWh

# transform!(gcdi_KPI_reserve, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(gcdi_KPI_reserve, :mu)

# transform!(gcd_KPI_reserve, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(gcd_KPI_reserve, :mu)
# ;

In [10]:
# KPI_reserve[!,Not([:delivered_r_up_ratio, :delivered_r_dn_ratio])] == KPI_reserve_[!,Not([:delivered_r_up_ratio, :delivered_r_dn_ratio])]
# gcdi_KPI_reserve_[!,Not([:delivered_r_up_ratio, :delivered_r_dn_ratio])] == gcdi_KPI_reserve[!,Not([:delivered_r_up_ratio, :delivered_r_dn_ratio])]
# gcd_KPI_reserve_ == gcd_KPI_reserve
# # gcdi_KPI_reserve_ == gcdi_KPI_reserve_
# # KPI_reserve == KPI_reserve
# # gcd_KPI_reserve = gcd_KPI_reserve_
# # gcdi_KPI_reserve = gcdi_KPI_reserve_

In [11]:
function check_loq_value(df, field, value, accuracy)
    filter = (df[!,field] .> value) .& (df[!,field] .< Inf) .& (.!isnan.(df[!,field]))
    filter = (df[!,field].*filter .-value) .< accuracy
    println(all(filter))
    return df[.!filter,:]

end
# Check
accuracy = 0.000001
check_loq_value(KPI_reserve, :delivered_r_up_ratio, 1, accuracy)
check_loq_value(KPI_reserve, :delivered_r_dn_ratio, 1, accuracy)
;
# balance = (KPI_reserve.required_r_up_MW .-KPI_reserve.required_r_dn_MW) .- (KPI_reserve.delivered_r_up_MW .- KPI_reserve.delivered_r_dn_MW .+ KPI_reserve.LOL_MW.*(required.>=0))
# println(all(abs.(balance) .<= accuracy))
# Todo: re-calcualte balance balance


true
true


Row,configuration,day,iteration,hour,demand_MW,LOL_MW,demand_uc_MW,required_r_up_MW,required_r_dn_MW,production_MW,charge_MW,discharge_MW,delivered_r_up_MW,delivered_r_dn_MW,delivered_r_up_ratio,delivered_r_dn_ratio
,Symbol,Int64,Symbol,Int64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64,Float64,Float64,Float64


In [13]:
to_plot = gcdi_KPI[abs.(gcdi_KPI.Δobjective_value_relative_ref_conf) .>0.001,:]
plot(to_plot, y = :Δobjective_value, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)

┌ Warning: attempting to remove probably stale pidfile
│   path = "/home/ubuntu/.jlassetregistry.lock"
└ @ Pidfile ~/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260


data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, xaxis4, yaxis, yaxis2, yaxis3, and yaxis4"

In [14]:
to_plot = gcdi_KPI[abs.(gcdi_KPI.Δobjective_value_relative_ref_conf) .>0.001,:]
plot(to_plot, y = :Δobjective_value_relative_ref_conf, x = :ENS_MWh, mode = "markers", facet_col = :day, color= :configuration, text =:iteration)

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, legend, margin, template, xaxis, xaxis2, xaxis3, xaxis4, yaxis, yaxis2, yaxis3, and yaxis4"

In [15]:
p1 = plot(gcd_KPI, x = :mu, y = :EOV, color = :day)
p2 = plot(gcd_KPI, x = :mu, y = :EENS, color = :day)
p3 = plot(gcd_KPI, x = :mu, y = :ECUR, color = :day)
# p3 = plot(KPI, x = :mu, y = :LOLE, color = :day)
[p1 p2]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [16]:
p1 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_up_ratio, color = :day)
p2 = plot(gcd_KPI_reserve, x = :mu, y = :delivered_r_dn_ratio, color = :day)
p3 = plot(gcd_KPI_reserve, x = :mu, y = :E_delivered_r_up_MWh, color = :day)
p4 = plot(gcd_KPI_reserve, x = :mu, y = :E_deliverered_dn_MWh, color = :day)
# [p1 p2; p3 p4]
[p1 p3]

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [17]:
plot(gcd_KPI_reserve, x = :mu, y = :E_required_r_dn_MWh, color = :day)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields legend, margin, template, xaxis, and yaxis"

In [18]:
plot(gcdi_KPI[gcdi_KPI.day .== 228,:], x = :mu, y = :ENS_MWh, kind = "box",  Layout(boxmode="group"))

data: [
  "box with fields type, x, xaxis, y, and yaxis"
]

layout: "layout with fields boxmode, legend, margin, template, xaxis, and yaxis"

In [19]:
# parse_configuration_to_mu(x) = !isnothing(match(r"base_ramp_storage_envelopes_up_(\w+)_dn_(\w+)", string(x))) ? parse(Float64, replace(match(r"base_ramp_storage_envelopes_up_(\w+)_dn_(\w+)", string(x))[1], "_" => ".")) : 1.1
# to_plot = transform(s_ed_scalar, :configuration .=> ByRow(x -> parse_configuration_to_mu(x)) .=> :mu)
# sort!(to_plot, :mu)
plot(gcdi_KPI, y = :ENS_MWh, x = :mu, facet_col = :day, color = :iteration)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [20]:
plot(gcdi_KPI, y = :Δobjective_value_relative_ref_conf, x = :mu, facet_col = :day, color = :iteration)

data: [
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and y

In [21]:
# plot(gdf_LOL_CUR, x = :CUR_MWh, y = :ENS_MWh, facet_col = :configuration, text = :iteration, color= :day, mode="markers")

In [22]:
# plot(LOL_CUR, x = :LOLE, y = :EENS, facet_col = :day, group = :configuration, marker_color = :EOV_M, mode = "markers", marker=attr(showscale=true))
# configs
plot(gcd_KPI, x = :ECUR, y = :EENS, facet_col = :day, mode = "markers", text = :day, color=:configuration) # marker=attr(showscale=true, coloraxis="coloraxis", color=:day

data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, text, type, x, xaxis, y, and yaxis",
  

In [23]:
# plot(LOL_CUR, x = :EOV, y = :EENS, text = :configuration, color = :configuration, mode = "markers")
plot(gcd_KPI, x = :EOV, y = :EENS, color = :configuration, mode = "markers", facet_col = :day)


data: [
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgroup, marker, mode, name, showlegend, type, x, xaxis, y, and yaxis",
  "scatter with fields legendgro

In [24]:
group_by = [:configuration, :day]
supply_uc, demand_uc = calculate_supply_demand(s_uc, union([:hour, :resource], group_by))
supply_ed, demand_ed = calculate_supply_demand(s_ed, union([:hour, :resource, :iteration], group_by))
commit_uc = combine(groupby(s_uc.generation, [:resource, :configuration, :day, :hour]), :commit => sum => :commit)
commit_ed = combine(groupby(s_ed.generation, [:resource, :configuration, :day, :iteration, :hour]), :commit => sum => :commit)
;

In [25]:
day_ = 228
iteration_ = :demand_48

:demand_48

### Checking reserve delivey

In [26]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
KPI_reserve_ = KPI_reserve[(KPI_reserve.configuration .== config_) .& (KPI_reserve.day .== day_) .& (KPI_reserve.iteration .== iteration_), :]
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_up_MW, name = "required_r_up_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_up_MW, name = "delivered_r_up_MW")]
p1 = plot(p, Layout(yaxis_title="reserve up MW", xaxis_title="hour"))
p = [scatter(KPI_reserve_, x=:hour, y=:required_r_dn_MW, name = "required_r_dn_MW"),scatter(KPI_reserve_, x=:hour, y=:delivered_r_dn_MW, name = "delivered_r_dn_MW")]
p2 = plot(p, Layout(yaxis_title="reserve dn MW", xaxis_title="hour"))
[p1 p2]

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

### Checking dispatch

In [27]:
day_ = 228
iteration_ = :demand_86

:demand_86

In [28]:
# Checking solutions are different
scalar = s_ed.scalar[(s_ed.scalar.day .== day_) .& (s_ed.scalar.iteration .== iteration_),:]
(scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_1_dn_1, :objective_value] - scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_dn_0, :objective_value]) / scalar[scalar.configuration .== :base_ramp_storage_envelopes_up_0_dn_0, :objective_value]

1×1 Matrix{Float64}:
 -0.9886223252438239

In [29]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [30]:
config_ = :base_ramp_storage_envelopes_up_1_dn_1
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [31]:
config_ = :base_ramp_storage_envelopes_up_0_dn_0
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [32]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [33]:
config_ = :base_ramp_storage_envelopes_up_1_dn_1
supply_ed_ = supply_ed[(supply_ed.configuration .== config_) .& (supply_ed.day .== day_) .& (supply_ed.iteration .== iteration_), :]
demand_ed_ = demand_ed[(demand_ed.configuration .== config_) .& (demand_ed.day .== day_) .& (demand_ed.iteration .== iteration_), :]
plot_supply_demand(supply_ed_, demand_ed_, string(config_))


data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [34]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]


data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

### Studying oscilations

In [35]:
day_ = 259
iteration_ = :demand_70

:demand_70

In [36]:
to_plot = commit_ed[(commit_ed.configuration .==  :base_ramp_storage_envelopes_up_0_5_dn_0_5) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar", title = "mu = 0.6")
to_plot = commit_ed[(commit_ed.configuration .==  :base_ramp_storage_envelopes_up_0_6_dn_0_6) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p2 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar" , title = "mu = 0.6")
[p1 p2]

data: [
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, title, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [37]:
config_ = :base_ramp_storage_envelopes_up_0_5_dn_0_5
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [38]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"

In [39]:
config_ = :base_ramp_storage_envelopes_up_0_6_dn_0_6
supply_uc_ = supply_uc[(supply_uc.configuration .== config_) .& (supply_uc.day .== day_), :]
demand_uc_ = demand_uc[(demand_uc.configuration .== config_) .& (demand_uc.day .== day_), :]
plot_supply_demand(supply_uc_, demand_uc_, string(config_))

data: [
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis",
  "scatter with fields line, mode, name, stackgroup, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, margin, template, title, xaxis1, xaxis2, yaxis1, and yaxis2"

In [40]:
to_plot = commit_ed[(commit_ed.configuration .== config_) .&  (commit_ed.day .== day_) .&  (commit_ed.iteration .== iteration_),:] 
p1 = plot(to_plot, x = :hour, y = :commit, color = :resource, kind = "bar")

to_plot = s_ed.storage[(s_ed.storage.configuration .== config_) .&  (s_ed.storage.day .== day_) .&  (s_ed.storage.iteration .== iteration_),:]
p2 = plot(to_plot, x = :hour , y = :SOE_MWh, color = :resource, kind = "bar", Layout(barmode="stack")) 
[p1 p2]

data: [
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis",
  "bar with fields legendgroup, marker, name, showlegend, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields margin, template, xaxis1, xaxis2, yaxis1, and yaxis2"